## Stable Matching Algorithm
Remember: stable matching mechanism ensures that no pair of student and project would prefer each other over their current matches.  


In [3]:
import pandas as pd
from collections import defaultdict

In [23]:
### Preparation

# Load the student and project data from Excel files
students_df = pd.read_excel('test_data/td_1_students.xlsx')
projects_df = pd.read_excel('test_data/td_1_projects.xlsx')

# Extract lists of students and projects
students = students_df['student_names'].tolist()
projects = projects_df['project_name'].tolist()

# Map each project to its capacity
project_capacity = projects_df.set_index('project_name')['max_students'].to_dict()

# Initialize dictionaries for student and project preferences
student_prefs = {}
project_prefs = {}

# Extract student preferences
for _, row in students_df.iterrows():
    student_prefs[row['student_names']] = row[1:].dropna().tolist()

# Extract project preferences
for _, row in projects_df.iterrows():
    # Assuming the first two columns are 'project_name' and 'max_students'
    project_prefs[row['project_name']] = row[2:].dropna().tolist()

### Preparation chunk:
- Load data, extract names of students and projects into lists and define project capacities
- Initialize dictionaries to store student and project preferences
- Loop extracting student and project preferences from the Excel files, ignoring NAs and skipping the first column (assumed to be a name or an ID)

In [22]:
### Matching algorithm

# Initialize matching and availability:
matches = {} # maps each student to their assigned project
project_assignments = defaultdict(list) # keys are project names and values are lists of students assigned to each project: dinamically updated

# Iteratively assign students to projects based on preferences and capacity
while len(matches) < len(students):
    for student in students: # loop continues until all students have been assigned to a project
        if student not in matches: # iterates over each student who hasn't been matched yet
            for project in student_prefs[student]: # goes through each student's project preferences in order from most preferred to least preferred
                if len(project_assignments[project]) < project_capacity[project] and student in project_prefs[project]: # check capacities and preferences
                    matches[student] = project
                    project_assignments[project].append(student)
                    break
                else:
                    # Handling over-subscription with bidirectional preference consideration
                    # At capactiy: evaluates if the new student could be more preferred compared to current assignees
                    if student in project_prefs[project]:
                        current_assignees = project_assignments[project]
                        # Include the new student for comparison while respecting project preferences
                        all_prefs = [s for s in project_prefs[project] if s in current_assignees + [student]]
                        preferred_assignees = sorted(current_assignees + [student], key=lambda x: all_prefs.index(x))[:project_capacity[project]] # ChatGPT suggestion
                        
                        if student in preferred_assignees: # if the new student is more preferred than the current assignees, adjust
                            new_assignees = preferred_assignees
                            for s in current_assignees:
                                if s not in new_assignees:
                                    project_assignments[project].remove(s)
                                    del matches[s]
                            if student not in project_assignments[project]:
                                project_assignments[project].append(student) # update assignments
                                matches[student] = project
                            break

# Print the final student-project matches
for student, project in matches.items():
    print(f"{student} is matched with {project}")

s1 is matched with p3
s2 is matched with p2
s5 is matched with p1
s6 is matched with p3
s7 is matched with p3
s8 is matched with p1
s9 is matched with p2
s3 is matched with p1
s4 is matched with p2


### Matching algorithm chunk:
Initial Setup:
- `matches`: A dictionary that will map each student to their assigned project.
- `project_assignments`: A `defaultdict(list) where keys are project names, and values are dynamically updated lists of students assign´d to each project.

Iterative Assignment Process:
- Continue Until All Are Matched: The algorithm runs until every student is matched with a project (`while len(matches) < len(students`)).
- Iterate Over Unmatched Students: For each student not yet assigned to a project (`if student not in matches`), the algorithm:
- Respect Student Preferences: It iterates through the student's project preferences in their order of preference, from most to least preferred (`for project in student_prefs[student]`).
- Check Project Capacity and Preferences: Before assigning a student to a project, it checks if the project still has capacity (`len(project_assignments[project]) < project_capacity[project]`) and if the student is among the project's preferences (`student in project_prefs[project]`).
- Assign Student to Project: If conditions are met, the student is assigned to the project, and both matches and project_assignments are updated accordingly.

Handle Over-Subscription:
- If a project is at capacity, the algorithm evaluates if the new student could be more preferred compared to current assignees, considering the project's preference list.
- It includes the new student for comparison alongside current assignees and identifies who the project would prefer most within its capacity limit.
- If the new student is more preferred, adjustments are made: less preferred students are removed from the project assignment, and the new student is added.

Output:
- Print final assignments